# MOKAS ASYMMETRIC CREEP IN BUBBLES

This code analysis a sequence of images of a bubble expansion under perpendicular and in-plane field.

The files (as images or a movie) are saved in folders with a specific pattern 

In [1]:
import numpy as np
import pandas as pd
import run_creep_on_bubbles as run_creep
import iniConnector as iniC
from IPython.display import display, Math
import matplotlib.pylab as plt
from ipywidgets import interact

## Choose your visualization library and the colormap here

For Matplotlib:
    https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html   
For Bokeh:
    https://docs.bokeh.org/en/latest/docs/reference/palettes.html
    
The colormap can also be chosen later.

In [2]:
#visualization_library = 'mpl'
visualization_library = 'bokeh'

if visualization_library == 'bokeh':
    from bokeh.io import push_notebook, show, output_notebook, export_png
    from bokeh.layouts import column
    from bokeh.plotting import figure
    from bokeh.models import (ColumnDataSource, Slider, Range1d, 
                                Div, LinearAxis, Legend, LegendItem, DataRange1d,
                                 CheckboxGroup, CustomJS)
    output_notebook()
    #colormap = 'Pastel1_9'
    colormap = 'Turbo256'
    
elif visualization_library == 'mpl':
    colormap = plt.cm.hsv
    #colormap = plt.cm.gist_ncar

backgroud_color = 'black'

Loading BokehJS ...

# Data properties

In [3]:
# Set if the data come from an experiment or a simulation
isExperiment = True
# Material name
material = "MeinzMAS185"
material_full = "$MeinzMAS185 - Meas$"  # To be shown in image's title

depinning_field = 25.6 # value in mT
frame_rate = 3.33 # frames/s
microns_per_pixel = 0.39

Define the main (root) folder containing the data to be analysed.   
Subdirectories are later created with in-plane fields definition.   
**Pattern** is the scheme to detect the sequence of images

In [9]:
root_dir = "/home/mokas/Meas/Creep/MeinzMAS185/run1"
#root_dir = "/home/gf/Meas/Creep/PtCoPt/M2/"
pattern = "seq1_*.png"

## Filtering & cropping
# Images require some filtering before the analysis.
# Usually a gaussian filter is fine, with a half-width of 1-2 pixels

# Gaussian filtering
filtering, sigma = "gauss", 1.5

# Crop image
#imCrop = [(0,0),(514,672)]
#imCrop = [(400,430),(660,660)]
imCrop = None #[(450,450),(620,620)]

# Delete small events as a percentage of the largest cluster found in the image
erase_small_events_percent = 2

# The number of frames to skip to plot the DW displacement
step_in_frames = 10

# Scaling of center of mass plotting (range: 1-5)
# When plotting the rescaled contours
# scale the data for the center of mass
center_of_mass_scaling = 1.5

# Pixel switch detection
# Kernel_half_width_of_ones
# if 3:
# ___|^^^
kernel_half_width_of_ones = 5
gray_threshold = 30


In [7]:
# Plot the cropped image
#%matplotlib notebook
#mainDir = "/home/mokas/Meas/Creep/MeinzMAS185/run2"
#mainDir = root_dir
#run_creep.show_cropped_image(mainDir, pattern, imCrop, figsize=(9,5))

# Applied fields

### Define the in-plane and out-of-plane fields

### 1. Out of plane fields

In [10]:
Bz_A = 4 # OoP field in V
Bz_unit = "A"
Bz_A_label = "%.3f" % Bz_A

# Conversion to mT
# Field (mT) = 14.37 * pulse intensity (A)
Bz_A_to_mT = 1.7
Bz_mT = Bz_A_to_mT * Bz_A

### 2. In-plane fields

In [15]:
#Bx_s = [-650,-600,-550,-500,-450,-400,-350,-300,-200,-100,-50,
#        0,
#        50,100,200,300,350,400,450,500,550,600,650]
Bx_s = [0]
Bx_unit = "Oe"

#Bx_s_labels = []
#for bx in Bx_s:
#    _bx = str(abs(bx)).rjust(3,"0")
#    if bx <= 0:
#        Bx_s_labels.append("%sn" % _bx)
#    else:
#        Bx_s_labels.append("%sp" % _bx)

#Bx_s = np.array(Bx_s)
#Bx_s_labels = np.array(Bx_s_labels)
# N. of measure (to be used in the pattern of the folder's name)
#n_measures = [1]
#n_measures = np.array(n_measures)
# Pulse durations in s
pulse_durations = [796]
pulse_durations = np.array(pulse_durations)

firstImage_s = [1] * len(Bx_s)
lastImage_s = [2647]
firstImage_s = np.array(firstImage_s)
lastImage_s = np.array(lastImage_s)

subDirs = ["cropSubtractPng"]

# Example of a subdir name
# subdir = "PtCoAuPt_3_2c-00d2-000nOe-0.780V-5.0s_1"

#for Bx_s_label,p,n in zip(Bx_s_labels,pulse_durations,n_measures):
#    subDir = "%s-%s%s-%sV-%.1fs_%i" % (material, Bx_s_label, Bx_unit, Bz_V_label, p, n)
#    subDirs.append(subDir)

subDirs = np.array(subDirs)
print(subDirs)
    
# We make a selection where the value at 0 in-plane field is always included
# Possible numbers of plots are 
# 9 with 4 neg, and 4 pos
# 15 with 7 neg, and 7 pos
# 21 with 10 neg, and 10 pos
# 27 with 13 neg and 13 pos
# Selection of fewer images
# IT IS MANDATORY TO HAVE THE SAME POS AND NEG VALUES
#zero_field_position = (len(Bx_s) - 1)//2
#if True:
#    # Symmetric solution
#    pos_neg_values, step = 7, 2
#    assert len(Bx_s) >= pos_neg_values * step + 1
#    i0 = list(range(zero_field_position-step,0,-step))
#    i0 = i0[::-1]
#    i1 = list(range(zero_field_position+step,zero_field_position+2*pos_neg_values+1,step))
#    indexes = i0 + [zero_field_position] + i1 
#    Bx_s = Bx_s[indexes]
#    Bx_s_labels = Bx_s_labels[indexes]
#    n_measures = n_measures[indexes]
#    pulse_durations = pulse_durations[indexes]
#    firstImage_s = firstImage_s[indexes]
#    lastImage_s = lastImage_s[indexes]
#    subDirs = subDirs[indexes]


['cropSubtractPng']


In [16]:
Bx_s

[0]

# Initialize the main code and update internal variables

#### (Do not touch anything below in the next two cells)

In [17]:
bubbles = run_creep.Creep(Bz=Bz_mT, gray_threshold=gray_threshold)
# Set if the analysis is for an experiment or a simulation
bubbles.measData = iniC.connect_to("",Bz=Bz_mT,isExperiment=isExperiment)

In [18]:
bubbles.Bz_mT = Bz_mT
bubbles.rootDir = root_dir
bubbles.imParameters = {}
bubbles.full_title = material_full + " - " + "$B_z = %i mT$" % Bz_mT
bubbles.Bx_s = Bx_s
bubbles.Bx_unit = Bx_unit
bubbles.step_in_frames = step_in_frames
bubbles.microns_per_pixel = microns_per_pixel
bubbles.frame_rate = frame_rate
bubbles.center_of_mass_scaling = center_of_mass_scaling

# Image Parameters
bubbles.imParameters['pattern'] = pattern
bubbles.imParameters['resize_factor'] = None
bubbles.imParameters['filtering'] = filtering
bubbles.imParameters['sigma'] = sigma
bubbles.imParameters['kernel_half_width_of_ones'] = kernel_half_width_of_ones
bubbles.imParameters['imCrop'] = imCrop
bubbles.imParameters['erase_small_events_percent'] = erase_small_events_percent


# Measurement parameters
bubbles.Bz_s = [Bz_mT]
bubbles.Bz_s_labels = [str(Bz_mT)]
bubbles.Bx_s = Bx_s
bubbles.Bx_s_labels = [str(_bx) for _bx in Bx_s]
bubbles.Bz_unit = Bz_unit
bubbles.Bx_unit = Bx_unit

bubbles.step_in_frames = step_in_frames
bubbles.gray_threshold = gray_threshold
bubbles.measData.varsBx = {}
max_n_images = 1
for i, Bx in enumerate(Bx_s):
    bubbles.measData.varsBx[Bx] = {}
    bubbles.measData.varsBx[Bx]['firstIm'] = firstImage_s[i]
    bubbles.measData.varsBx[Bx]['lastIm'] = lastImage_s[i]
    bubbles.measData.varsBx[Bx]['subDir'] = subDirs[i]
    bubbles.measData.varsBx[Bx]['pulse_duration'] = pulse_durations[i]
    n = lastImage_s[i] - firstImage_s[i]
    if n > max_n_images:
        max_n_images = n
bubbles.max_n_images = (max_n_images//10 + 1) * 10
bubbles.all_contours = {}
bubbles.all_centers = {}

In [19]:
# This cell can be run many times

bubbles.prepare_plots(colormap=colormap, visualization_library=visualization_library)

1920 1080
Preparing plotsIt is better to choose 9, 15, 21 or 27 field values

Done
# ##################################################
Bx = 0 Oe, Bz = 6 mT


TypeError: cannot unpack non-iterable NoneType object

## Fig1. Sequence of domain wall positions in color

In [22]:
bubbles.plot_figure('colorImage', visualization_library='bokeh')

In [1]:
bubbles.imArray_collector


NameError: name 'bubbles' is not defined

In [35]:
imArray = bubbles.imArray_collector[-30]
imArray.showRawAndCalcEvents(10, figsize=(6,2))

KeyError: -30

## Fig2. Histogram fraction of area covered for each image

In [ ]:
bubbles.plot_figure('histogram', visualization_library='bokeh')

## Fig. 3 Contours plots

In [ ]:
bubbles.plot_figure('contours', visualization_library='bokeh')

In [ ]:
bubbles.plot_figure('contours_rescaled', visualization_library='bokeh')

In [ ]:
bubbles.plot_figure('distance_from_center', visualization_library='bokeh')
#if visualization_library == 'mpl':
#    creep_data.fig5
#elif visualization_library == 'bokeh':
    #display(Math(r"%s" % creep_data.full_title),)
#    show(creep_data.fig5, notebook_handle=True)
    # to save the figure
    #export_png(creep_data.fig5, filename = 'fig5.png')

In [ ]:
bubbles.plot_figure('DW displacement', visualization_library='bokeh')

## DW velocity as a function of the angle (0 degrees are along the orizontal)

In [ ]:
df = bubbles.velocities.T

fig = figure(plot_height=400, plot_width=400, title="Velocity vs in-plane fields",
              tools="crosshair,pan,box_zoom,save,reset",
              x_range=[-180,180])
#df.reset_index(inplace=True)
#df.rename(columns = {'index': 'Bx'}, inplace=True)
#c0 = fig.circle(x=df.index, y=df[0.0], color='blue', legend_label="0 degree")
#c1 = fig.circle(x=df.index, y=df[-180], color='red', legend_label="180 degree")
c0 = fig.circle(x=df.index, y=df[0.0], color='blue')
c1 = fig.circle(x=df.index, y=df[-180], color='red')



max_y = max(df[0.0]) * 1.1
_max_y = max(df[-180.0]) * 1.1
if _max_y > max_y:
    max_y = _max_y
min_x = min(Bx_s) * 1.1
max_x = max(Bx_s) * 1.1
fig.x_range = Range1d(min_x, max_x)
fig.y_range = Range1d(start=0, end=max_y)
fig.xaxis.axis_label = "In-plane field (Oe)"
fig.yaxis.axis_label = "pixel/s"
# Create 2nd LHS y-axis
fig.extra_y_ranges['um/s'] = Range1d(start=0, end=max_y*microns_per_pixel)
fig.extra_x_ranges['mT'] = Range1d(start=min_x*1e-1, end=max_x*1e-1)
fig.add_layout(LinearAxis(y_range_name='um/s', axis_label='microns/s'), 'right')
fig.add_layout(LinearAxis(x_range_name='mT', axis_label='In-plane field (mT)'), 'above')
#fig.legend.location = "top_center"
angle_slider = Slider(title="angle", value=0, start=-180, end=180, step=15)
out = column(fig, angle_slider)
handle = show(fig, notebook_handle=True)


def update_velocity(angle):

    # Get the current slider values
    a = int(angle)
    if a < 0:
        b = 180 + a
        a, b = b, a
    else:
        b = a -180
        
        # Generate the new curve
    
    c0.data_source.data['y'] = df[a]
    c1.data_source.data['y'] = df[b]
    new_legend_values = ["%i degree" % a for angle in [a,b]]
    #for legend_item, new_value in zip(fig.legend.items, new_legend_values):
    #    legend_item.label['value'] = new_value
    #print(fig.legend.items[0].label)
    #print(fig.legend.items[1].label)
    #li = fig.legend.items
    #fig.legend.items = [(new_legend_values[0], [c0]),
    #                    (new_legend_values[1], [c1])]
    #fig.legend.items[0].label = new_legend_values[0]
    #fig.legend.items[1].label = new_legend_values[1]
    #legend = Legend(items=[
    #        LegendItem(label=new_legend_values[0], renderers=[c0], index=0),
    #        LegendItem(label=new_legend_values[1], renderers=[c1], index=0),
    #            ])
    
    push_notebook(handle=handle)
    
interact(update_velocity, angle=(-180, 180, 15))
